In [144]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.graph_objects as go

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [145]:
df = pd.read_csv('input/all_fuels_data.csv')
df.head()

,ticker,commodity,date,open,high,low,close,volume
0,CL=F,Crude Oil,2000-08-23,31.950001,32.799999,31.950001,32.049999,79385
1,CL=F,Crude Oil,2000-08-24,31.900000,32.240002,31.400000,31.629999,72978
2,CL=F,Crude Oil,2000-08-25,31.700001,32.099998,31.320000,32.049999,44601
3,CL=F,Crude Oil,2000-08-28,32.040001,32.919998,31.860001,32.869999,46770
4,CL=F,Crude Oil,2000-08-29,32.820000,33.029999,32.560001,32.720001,49131


In [152]:
fuel_data = df[df['commodity']=='RBOB Gasoline'].reset_index()
fuel_data['date'] = pd.to_datetime(fuel_data['date'], errors='coerce')
fuel_data['year'] = fuel_data['date'].dt.year
fuel_data['month'] = fuel_data['date'].dt.month
fuel_data['day'] = fuel_data['date'].dt.day
fuel_data = fuel_data.drop(columns=['date', 'ticker', 'commodity', 'index', 'open', 'high', 'volume', 'low'])


In [153]:
fuel_data.head()

,close,year,month,day
0,0.8840,2000,11,1
1,0.8688,2000,11,2
2,0.8695,2000,11,3
3,0.8825,2000,11,6
4,0.8910,2000,11,7


In [154]:
train, test = train_test_split(fuel_data, test_size=0.2, shuffle=False)

In [155]:
X = fuel_data.drop(['close'], axis=1)  # замените 'целевая_переменная' на имя столбца с ценой бензина
y = fuel_data['close']

In [157]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Вывод размеров обучающего и тестового наборов
print("Размеры обучающего и тестового наборов:")
print("Обучающий набор:", X_train.shape, y_train.shape)
print("Тестовый набор:", X_test.shape, y_test.shape)

Размеры обучающего и тестового наборов:
Обучающий набор: (4723, 3) (4723,)
Тестовый набор: (1181, 3) (1181,)


In [158]:
reg = xgb.XGBRegressor(n_estimators=10000, learning_rate=0.01,
                       early_stopping_rounds=200)
reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

[0]	validation_0-rmse:0.73397	validation_1-rmse:0.70858
[100]	validation_0-rmse:0.35182	validation_1-rmse:0.34072
[200]	validation_0-rmse:0.19820	validation_1-rmse:0.19506
[300]	validation_0-rmse:0.14203	validation_1-rmse:0.14231
[400]	validation_0-rmse:0.11602	validation_1-rmse:0.11742
[500]	validation_0-rmse:0.10130	validation_1-rmse:0.10375
[600]	validation_0-rmse:0.09346	validation_1-rmse:0.09699
[700]	validation_0-rmse:0.08675	validation_1-rmse:0.09229
[800]	validation_0-rmse:0.08108	validation_1-rmse:0.08766
[900]	validation_0-rmse:0.07561	validation_1-rmse:0.08387
[1000]	validation_0-rmse:0.07291	validation_1-rmse:0.08233
[1100]	validation_0-rmse:0.07015	validation_1-rmse:0.08045
[1200]	validation_0-rmse:0.06744	validation_1-rmse:0.07869
[1300]	validation_0-rmse:0.06485	validation_1-rmse:0.07688
[1400]	validation_0-rmse:0.06227	validation_1-rmse:0.07519
[1500]	validation_0-rmse:0.06068	validation_1-rmse:0.07400
[1600]	validation_0-rmse:0.05850	validation_1-rmse:0.07234
[1700]	va

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=200,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [160]:
y_pred = reg.predict(X_test)

# Оценка качества модели
mse = mean_squared_error(y_test, y_pred)
print("Среднеквадратичная ошибка на тестовом наборе:", mse)

Среднеквадратичная ошибка на тестовом наборе: 0.003247740194730301


In [161]:
X_test_sorted = X_test.sort_index()

# Получаем фактические и предсказанные значения
actual_prices = y_test.sort_index().values
predicted_prices = reg.predict(X_test_sorted)

# Создаем график с помощью Plotly
fig = go.Figure()

# Добавляем фактические цены
fig.add_trace(go.Scatter(x=X_test_sorted.index, y=actual_prices, mode='lines', name='Фактическая цена'))

# Добавляем предсказанные цены
fig.add_trace(go.Scatter(x=X_test_sorted.index, y=predicted_prices, mode='lines', name='Предсказанная цена'))

# Настройка макета графика
fig.update_layout(
    title='Фактическая и предсказанная цена бензина',
    xaxis_title='Индекс',
    yaxis_title='Цена',
    showlegend=True
)

# Отображаем график
fig.show()

In [105]:
X_test_sorted

,year,month,day
6,2000,9,8
7,2000,9,11
9,2000,9,13
10,2000,9,14
15,2000,9,21
...,...,...,...
5925,2024,4,5
5935,2024,4,19
5939,2024,4,25
5943,2024,5,1


In [106]:
X_test_sorted.to_csv('data/X_test_sorted.csv', index=False)

In [107]:
X_test_sorted = pd.read_csv('data/X_test_sorted.csv')
X_test_sorted

,year,month,day
0,2000,9,8
1,2000,9,11
2,2000,9,13
3,2000,9,14
4,2000,9,21
...,...,...,...
1185,2024,4,5
1186,2024,4,19
1187,2024,4,25
1188,2024,5,1


In [162]:
predicted_prices

array([0.88630915, 0.9128865 , 0.8814933 , ..., 2.8004432 , 2.7204237 ,
       2.5593274 ], dtype=float32)

In [163]:
df_actual_prices = pd.DataFrame(predicted_prices, columns=['Predicted_Prices'])

# Сохраняем DataFrame в CSV файл
df_actual_prices.to_csv('data/predicted_prices_gasoline.csv', index=False)


In [164]:
predicted_prices = pd.read_csv('data/predicted_prices_gasoline.csv')
predicted_prices

,Predicted_Prices
0,0.886309
1,0.912887
2,0.881493
3,0.820469
4,0.750384
...,...
1176,2.659823
1177,2.857751
1178,2.800443
1179,2.720424


In [165]:
actual_prices

array([0.87199998, 0.91970003, 0.90600002, ..., 2.72869992, 2.68540001,
       2.55509996])

In [169]:
df_actual_prices = pd.DataFrame(actual_prices, columns=['Actual_Prices'])

# Сохраняем DataFrame в CSV файл
df_actual_prices.to_csv('data/actual_prices_gasoline.csv', index=False)

In [172]:
actual_prices = pd.read_csv('data/actual_prices_gasoline.csv')
actual_prices

,Actual_Prices
0,0.8720
1,0.9197
2,0.9060
3,0.8220
4,0.7645
...,...
1176,2.5864
1177,2.7839
1178,2.7287
1179,2.6854


In [173]:
# actual_prices = y_test.sort_index().values
# predicted_prices = reg.predict(X_test_sorted)

# Создаем график с помощью Plotly
fig = go.Figure()

# Добавляем фактические цены
fig.add_trace(go.Scatter(x=X_test_sorted.index, y=actual_prices['Actual_Prices'], mode='lines', name='Фактическая цена'))

# Добавляем предсказанные цены
fig.add_trace(go.Scatter(x=X_test_sorted.index, y=predicted_prices['Predicted_Prices'], mode='lines', name='Предсказанная цена'))

# Настройка макета графика
fig.update_layout(
    title='Фактическая и предсказанная цена бензина',
    xaxis_title='Индекс',
    yaxis_title='Цена',
    showlegend=True
)

# Отображаем график
fig.show()

In [12]:
# Создаем DataFrame для даты предсказания

pred_date = {
    'year': 2024,
    'month': 5,
    'day': 27
}
next_month_features = pd.DataFrame([pred_date])

# Делаем предсказание
next_month_price = reg.predict(next_month_features)[0]
print("Предсказанная цена:", next_month_price)


Предсказанная цена: 1.7722687


In [13]:
import joblib

# Сохраняем модель в файл
joblib.dump(reg, 'models/nature_gaz_xgboost_model.pkl')

['models/nature_gaz_xgboost_model.pkl']

In [141]:
actual_prices = pd.read_csv('data/actual_prices.csv')
predicted_prices = pd.read_csv('data/predicted_prices.csv')
X_test_sorted = pd.read_csv('data/X_test_sorted.csv')

# Создаем график с помощью Plotly
fig = go.Figure()

# Добавляем фактические цены
fig.add_trace(go.Scatter(x=X_test_sorted.index, y=actual_prices['Actual_Prices'], mode='lines', name='Фактическая цена'))

# Добавляем предсказанные цены
fig.add_trace(go.Scatter(x=X_test_sorted.index, y=predicted_prices['Predicted_Prices'], mode='lines', name='Предсказанная цена'))

# Настройка макета графика
fig.update_layout(
    title='Фактическая и предсказанная цена бензина',
    xaxis_title='Индекс',
    yaxis_title='Цена',
    showlegend=True
)

# Отображаем график
fig.show()